In [1]:
import pandas as pd
import geopandas as gpd

/home/data/work/Fatemeh/DEiA-2-main/.direnv/python-3.10.1/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
CRS = "EPSG:32733"

In [3]:
facilities = pd.read_csv('nl_osm_locations.csv')
facilities = gpd.GeoDataFrame(
    facilities, geometry=gpd.points_from_xy(facilities['lon'], facilities['lat'], crs=CRS)
).drop(columns=['lat', 'lon'])

In [4]:
neighborhoods = pd.read_csv('neighbourhood_to_coordinate.csv')
neighborhoods = gpd.GeoDataFrame(
    neighborhoods, geometry=gpd.points_from_xy(neighborhoods['longitude'], neighborhoods['latitude']), crs=CRS
).drop(columns=['latitude', 'longitude'])
neighborhoods

,municipality,neighbourhood,play_demand,a_00_14,geometry
0,Appingedam,Appingedam-Centrum,2,230,POINT (6.851 53.329)
1,Appingedam,Appingedam-West,3,480,POINT (6.831 53.309)
2,Appingedam,Appingedam-Oost,7,900,POINT (6.859 53.307)
3,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,-1,65,POINT (6.831 53.309)
4,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,-2,10,POINT (6.859 53.307)
...,...,...,...,...,...
13515,Molenlanden,Waal-Dorp,-2,20,POINT (4.901 51.938)
13516,Molenlanden,Wijngaarden-Buitengebied,-2,5,POINT (4.764 51.845)
13517,Molenlanden,Kern-Dorp,0,60,POINT (4.764 51.845)
13518,Molenlanden,Lintbebouwing-Oost,-2,40,POINT (4.764 51.845)


In [5]:
def join_nearest_type(type_):
    return gpd.sjoin_nearest(neighborhoods, facilities.loc[facilities['type'] == type_], distance_col='distance')

In [6]:
def join_nearest_types(types):
    dfs = [join_nearest_type(t) for t in types]
    return pd.concat(dfs, axis=0, ignore_index=True)

In [7]:
facility_types = list(facilities['type'].unique())
facility_types

['nl_sports_centre',
 'nl_sports_hall',
 'nl_kindergarten',
 'nl_childcare',
 'nl_fitness_station',
 'nl_pitch',
 'nl_swimming_pool',
 'nl_stadium',
 'nl_college',
 'nl_fitness_centre',
 'nl_university',
 'nl_park',
 'nl_golf_course',
 'nl_horse_riding',
 'nl_track']

In [8]:
joined = join_nearest_types(facility_types)
joined

,municipality,neighbourhood,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam,Appingedam-Centrum,2,230,POINT (6.851 53.329),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.048259
1,Appingedam,Appingedam-West,3,480,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.071204
2,Appingedam,Appingedam-Oost,7,900,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.046349
3,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,-1,65,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.071204
4,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,-2,10,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.046349
...,...,...,...,...,...,...,...,...,...,...,...,...
202255,Hoeksche Waard,Zuid-Beijerland Buitengebied,-1,70,POINT (4.369 51.751),6155,6155,node/7653181888,Diomedon,NaN,nl_track,0.176187
202256,Hoeksche Waard,Goudswaard Woonkern,1,280,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,0.218013
202257,Hoeksche Waard,Goudswaard Buitengebied,-1,55,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,0.218013
202258,Hoeksche Waard,Piershil Woonkern,1,200,POINT (4.316 51.796),6155,6155,node/7653181888,Diomedon,NaN,nl_track,0.216032


In [9]:
joined['distance'] = round(joined['distance'] * 110.486, 2)
joined

,municipality,neighbourhood,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam,Appingedam-Centrum,2,230,POINT (6.851 53.329),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.33
1,Appingedam,Appingedam-West,3,480,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,7.87
2,Appingedam,Appingedam-Oost,7,900,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.12
3,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,-1,65,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,7.87
4,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,-2,10,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.12
...,...,...,...,...,...,...,...,...,...,...,...,...
202255,Hoeksche Waard,Zuid-Beijerland Buitengebied,-1,70,POINT (4.369 51.751),6155,6155,node/7653181888,Diomedon,NaN,nl_track,19.47
202256,Hoeksche Waard,Goudswaard Woonkern,1,280,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,24.09
202257,Hoeksche Waard,Goudswaard Buitengebied,-1,55,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,24.09
202258,Hoeksche Waard,Piershil Woonkern,1,200,POINT (4.316 51.796),6155,6155,node/7653181888,Diomedon,NaN,nl_track,23.87


In [10]:
joined.loc[joined['neighbourhood'] == 'Haren']

,municipality,neighbourhood,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
233,Groningen,Haren,6,1485,POINT (6.586 53.171),109,109,node/469191049,Groninger Kanovereniging,9752XJ,nl_sports_centre,1.52
3030,Oss,Haren,0,85,POINT (5.585 51.801),93,93,node/359101300,Den Hoge Graaf,5371DA,nl_sports_centre,6.01
13568,Groningen,Haren,6,1485,POINT (6.586 53.171),2020,2020,node/2842128528,Gymzaal,9471SE,nl_sports_hall,13.02
19807,Oss,Haren,0,85,POINT (5.585 51.801),2023,2023,node/2860266411,De Burcht,5427AM,nl_sports_hall,23.89
27153,Groningen,Haren,6,1485,POINT (6.586 53.171),2322,2322,node/2740107533,Het Helpernest,9721AJ,nl_kindergarten,2.97
36661,Oss,Haren,0,85,POINT (5.585 51.801),2237,2237,node/2449891324,kinderdagverblijf 't Hooibergje,NaN,nl_kindergarten,7.34
40597,Groningen,Haren,6,1485,POINT (6.586 53.171),2970,2970,node/2748274225,Kinderopvang Groningen,9725AC,nl_childcare,4.35
44839,Oss,Haren,0,85,POINT (5.585 51.801),3123,3123,node/2973296424,Stip en Stap,6658KG,nl_childcare,11.29
54578,Groningen,Haren,6,1485,POINT (6.586 53.171),3490,3490,node/7335004779,NaN,NaN,nl_fitness_station,3.63
59044,Oss,Haren,0,85,POINT (5.585 51.801),3577,3577,node/9151171597,NaN,NaN,nl_fitness_station,12.46


In [11]:
facilities.loc[facilities['name'] == 'Dreamfit']

,Unnamed: 0,id,name,addr:postcode,type,geometry
5166,5166,node/2740108333,Dreamfit,9721AJ,nl_fitness_centre,POINT (6.581 53.197)


## Categorize

In [12]:
category = {
    "playground": ["nl_kindergarten", "nl_childcare"],
    "sport": [
        "nl_sports_centre",
        "nl_sports_hall",
        "nl_fitness_station",
        "nl_fitness_centre",
        "nl_stadium",
        "nl_swimming_pool",
        "nl_golf_course",
        "nl_horse_riding",
        "nl_university",
        "nl_track",
        "nl_college",
        "nl_pitch"
    ],
    "park": ["nl_park"]
}

In [13]:
def categorize(type_):
    for c in category.keys():
        for t in category[c]:
            if t == type_:
                return c

In [14]:
categorize("nl_pitch")

'sport'

In [15]:
joined['category'] = joined.apply(lambda r: categorize(r['type']), axis=1)
joined

,municipality,neighbourhood,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance,category
0,Appingedam,Appingedam-Centrum,2,230,POINT (6.851 53.329),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.33,sport
1,Appingedam,Appingedam-West,3,480,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,7.87,sport
2,Appingedam,Appingedam-Oost,7,900,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.12,sport
3,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,-1,65,POINT (6.831 53.309),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,7.87,sport
4,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,-2,10,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.12,sport
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202255,Hoeksche Waard,Zuid-Beijerland Buitengebied,-1,70,POINT (4.369 51.751),6155,6155,node/7653181888,Diomedon,NaN,nl_track,19.47,sport
202256,Hoeksche Waard,Goudswaard Woonkern,1,280,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,24.09,sport
202257,Hoeksche Waard,Goudswaard Buitengebied,-1,55,POINT (4.278 51.794),6155,6155,node/7653181888,Diomedon,NaN,nl_track,24.09,sport
202258,Hoeksche Waard,Piershil Woonkern,1,200,POINT (4.316 51.796),6155,6155,node/7653181888,Diomedon,NaN,nl_track,23.87,sport


In [16]:
joined.to_csv("neighborhood_facilities_joined_all.csv")

## Minimum distance of each category

In [17]:
cs = list(joined.drop(columns='geometry').columns)
cs # The geometry column doesn't play well with aggregations

['municipality',
 'neighbourhood',
 'play_demand',
 'a_00_14',
 'index_right',
 'Unnamed: 0',
 'id',
 'name',
 'addr:postcode',
 'type',
 'distance',
 'category']

In [18]:
min_cat = joined.groupby([
    'neighbourhood', 
    'municipality', 
    'play_demand', 
    'a_00_14',
    'category'
], as_index=False)['distance'].min()
min_cat

,neighbourhood,municipality,play_demand,a_00_14,category,distance
0,'n Oaln Diek,Hellendoorn,0,140,park,33.14
1,'n Oaln Diek,Hellendoorn,0,140,playground,3.61
2,'n Oaln Diek,Hellendoorn,0,140,sport,1.93
3,'s Gravenland,Rotterdam,7,1230,park,10.85
4,'s Gravenland,Rotterdam,7,1230,playground,1.06
...,...,...,...,...,...,...
40438,vesten,Houten,-2,0,playground,2.18
40439,vesten,Houten,-2,0,sport,1.47
40440,weiden,Houten,2,285,park,5.10
40441,weiden,Houten,2,285,playground,0.77


In [19]:
min_cat.loc[min_cat['neighbourhood'] == 'Haren']

,neighbourhood,municipality,play_demand,a_00_14,category,distance
13638,Haren,Groningen,6,1485,park,4.62
13639,Haren,Groningen,6,1485,playground,2.97
13640,Haren,Groningen,6,1485,sport,1.52
13641,Haren,Oss,0,85,park,11.23
13642,Haren,Oss,0,85,playground,7.34
13643,Haren,Oss,0,85,sport,6.01


In [20]:
joined.to_csv("neighborhood_facilities_min_distance_category.csv")